In [29]:
# txt 형식의 json파일을 읽기
from pprint import pprint
import json
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
with open('data.txt') as f:
    json_data = json.load(f)
json_data

[{'movieCd': '20197336',
  'movieNm': '허슬러',
  'movieNmEn': 'Hustlers',
  'prdtYear': '2019',
  'openDt': '',
  'typeNm': '장편',
  'prdtStatNm': '개봉예정',
  'nationAlt': '미국',
  'genreAlt': '범죄,드라마',
  'repNationNm': '미국',
  'repGenreNm': '범죄',
  'directors': [{'peopleNm': '로렌 스카파리아'}],
  'companys': []},
 {'movieCd': '20190064',
  'movieNm': '산타 앤 컴퍼니',
  'movieNmEn': 'Christmas & Co',
  'prdtYear': '2017',
  'openDt': '',
  'typeNm': '장편',
  'prdtStatNm': '개봉예정',
  'nationAlt': '프랑스',
  'genreAlt': '판타지',
  'repNationNm': '프랑스',
  'repGenreNm': '판타지',
  'directors': [{'peopleNm': '알랭 샤바'}],
  'companys': []},
 {'movieCd': '20191323',
  'movieNm': '오렌지필름 기획전 - Fragile; 파손주의 ',
  'movieNmEn': '',
  'prdtYear': '',
  'openDt': '',
  'typeNm': '장편',
  'prdtStatNm': '기타',
  'nationAlt': '한국',
  'genreAlt': '',
  'repNationNm': '한국',
  'repGenreNm': '',
  'directors': [],
  'companys': []},
 {'movieCd': '20171561',
  'movieNm': '싱',
  'movieNmEn': 'SING',
  'prdtYear': '2016',
  'openDt': '',

In [21]:
import csv
csv_data = open('./movie.csv', 'w', encoding='utf-8')
csvwriter = csv.writer(csv_data)
count = 0
for kk in json_data:
    if count == 0:
        header = kk.keys()
        csvwriter.writerow(header)
        count += 1
    csvwriter.writerow(kk.values())
csv_data.close()

In [22]:
import pandas as pd
df = pd.read_csv("movie.csv")
df.head

,movieCd,movieNm,movieNmEn,prdtYear,openDt,typeNm,prdtStatNm,nationAlt,genreAlt,repNationNm,repGenreNm,directors,companys
0,20197336,허슬러,Hustlers,2019.0,NaN,장편,개봉예정,미국,"범죄,드라마",미국,범죄,[{'peopleNm': '로렌 스카파리아'}],[]
1,20190064,산타 앤 컴퍼니,Christmas & Co,2017.0,NaN,장편,개봉예정,프랑스,판타지,프랑스,판타지,[{'peopleNm': '알랭 샤바'}],[]
2,20191323,오렌지필름 기획전 - Fragile; 파손주의,NaN,NaN,NaN,장편,기타,한국,NaN,한국,NaN,[],[]
3,20171561,싱,SING,2016.0,NaN,단편,기타,헝가리,드라마,헝가리,드라마,[{'peopleNm': '크리스토프 데아크'}],[]
4,20191306,그녀의 대험한 유혹,NaN,NaN,NaN,기타,기타,NaN,NaN,NaN,NaN,[{'peopleNm': '아사기리 조'}],[]
5,20196781,우리를 갈라놓는 것들,Things That Do Us Part,2019.0,NaN,장편,개봉예정,한국,다큐멘터리,한국,다큐멘터리,[{'peopleNm': '임흥순'}],"[{'companyCd': '20137935', 'companyNm': '반달'}]"
6,20191322,러브 아일랜드,Love Island,NaN,NaN,장편,NaN,NaN,NaN,NaN,NaN,[{'peopleNm': '김형협'}],[]
7,20194781,평양을 즐겨요,Have Fun in Pyongyang,2018.0,NaN,장편,기타,프랑스,다큐멘터리,프랑스,다큐멘터리,[],[]
8,20195083,너의 여자친구,My Bossy Girl,2018.0,NaN,장편,개봉예정,한국,"코미디,멜로/로맨스,드라마",한국,코미디,[{'peopleNm': '이장희'}],[]
9,20196284,윤희에게,Moonlit Winter,2019.0,20191114.0,장편,개봉,한국,멜로/로맨스,한국,멜로/로맨스,[{'peopleNm': '임대형'}],"[{'companyCd': '20139335', 'companyNm': '영화사 달..."


In [26]:
features = ['genreAlt']
def combine_features(row):
    return ' '.join(row['genreAlt'].split(','))
# NaN을 빈 string로 채움
for feature in features:
    df[features] = df[features].fillna('')  # NaN을 ''로 변경하여 채워줌
df['combined_features'] = df.apply(combine_features, axis=1) # combine feature df에 추가
df['combined_features']

0            범죄 드라마
1               판타지
2                  
3               드라마
4                  
5             다큐멘터리
6                  
7             다큐멘터리
8    코미디 멜로/로맨스 드라마
9            멜로/로맨스
Name: combined_features, dtype: object

# 영화 제목을 이용해 비슷한 영화들을 찾는 경우

In [47]:
cv = CountVectorizer()  # creating new CountVectorizer() object
count_matrix = cv.fit_transform(df['combined_features'])
# feeding combined strings(movie contents) to CountVectorizer() object
cosine_sim = cosine_similarity(count_matrix)

# 함수
def get_title_from_index(index):
    return df['movieNm'][index]
def get_index_from_title(movieNm):
    return df.loc[df['movieNm'] == movieNm].index[0]

movie_user_likes = '싱'
movieCd = get_index_from_title(movie_user_likes)
similar_movies = list(enumerate(cosine_sim[movieCd]))
similar_movies
sorted_similar_movies = sorted(similar_movies, key=lambda x: x[1], reverse=True)[1:]
print('=====해당 영화와 비슷한 영화들=====')
for e in sorted_similar_movies:
    if e[1] < 0.5:
        break
    print(get_title_from_index(e[0]))

=====해당 영화와 비슷한 영화들=====
허슬러
너의 여자친구


# 영화의 모든 추가정보를 이용하여 가장 비슷한 영화를 찾는 경우